In [1]:
import sys
sys.path.insert(0, '/home/oscarwu/code/chemprop_developing')

import pickle as pkl
import numpy as np
import pandas as pd

In [2]:
# load an example
df_smi = pd.read_csv('esol.csv')
df_smi.iloc[[7]]

,smiles,logSolubility
7,CC#N,0.26


In [3]:
smiles = df_smi.iloc[[7]]['smiles'].values.tolist()

In [4]:
smiles

['CC#N']

In [7]:
from chemprop.features.utils import load_features, load_valid_atom_or_bond_features
from chemprop.models.mpn import mask_features_extra_batch, mask_features_extra

In [8]:
bond_features = [load_valid_atom_or_bond_features('esol_wb97xd_bond_RBF_features.pkl', smiles)[7]]

In [9]:
atom_descriptors = [load_valid_atom_or_bond_features('esol_wb97xd_atom_RBF_features.pkl', smiles)[7]]

In [10]:
atom_features[0].shape

NameError: name 'atom_features' is not defined

In [ ]:
atom_features

[array([[2.55145405e-004, 1.30985746e-003, 5.61677873e-003, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115],
        [1.90049994e-018, 8.02235393e-017, 2.82854289e-015, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115],
        [3.69512701e-008, 4.04123296e-007, 3.69169145e-006, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115],
        [2.03092857e-017, 7.65751884e-016, 2.41161786e-014, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115],
        [2.03092857e-017, 7.65751884e-016, 2.41161786e-014, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115],
        [2.03092857e-017, 7.65751884e-016, 2.41161786e-014, ...,
         2.95654057e-106, 8.55867443e-111, 1.98831132e-115]])]

In [ ]:
type(atom_features[0])

numpy.ndarray

In [11]:
mol_features = load_features("esol_wb97xd_molecule_features.csv")[7].reshape(1, -1)

In [12]:
mol_features.shape

(1, 20)

In [13]:
mol_features

array([[ 0.67249   ,  0.6725    ,  0.68771   ,  0.76879   ,  0.56122   ,
         0.56123   ,  0.57644   ,  0.65752   ,  0.53228   ,  0.53229   ,
         0.5475    ,  0.62858   ,  0.53228   ,  0.53229   ,  0.5475    ,
         0.62858   ,  0.45305768, -0.13823013,  3.8165    ,  2.179     ]])

In [14]:
type(mol_features)

numpy.ndarray

In [15]:
mol_features

array([[ 0.67249   ,  0.6725    ,  0.68771   ,  0.76879   ,  0.56122   ,
         0.56123   ,  0.57644   ,  0.65752   ,  0.53228   ,  0.53229   ,
         0.5475    ,  0.62858   ,  0.53228   ,  0.53229   ,  0.5475    ,
         0.62858   ,  0.45305768, -0.13823013,  3.8165    ,  2.179     ]])

In [16]:
mask_features_extra_batch([mol_features], [[True]*20], feature_length=1)

[array([[ 0.67249   ,  0.6725    ,  0.68771   ,  0.76879   ,  0.56122   ,
          0.56123   ,  0.57644   ,  0.65752   ,  0.53228   ,  0.53229   ,
          0.5475    ,  0.62858   ,  0.53228   ,  0.53229   ,  0.5475    ,
          0.62858   ,  0.45305768, -0.13823013,  3.8165    ,  2.179     ]])]

In [17]:
mask_features_extra_batch([mol_features], [[False]*20], feature_length=1)

[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0., -0.,  0.,  0.]])]

In [18]:
mask_features_extra_batch([mol_features], [[True] + [False]*19], feature_length=1)

[array([[ 0.67249,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
          0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
          0.     ,  0.     ,  0.     ,  0.     ,  0.     , -0.     ,
          0.     ,  0.     ]])]

In [19]:
mask_features_extra_batch([mol_features], [[False] + [True]*19], feature_length=1)

[array([[ 0.        ,  0.6725    ,  0.68771   ,  0.76879   ,  0.56122   ,
          0.56123   ,  0.57644   ,  0.65752   ,  0.53228   ,  0.53229   ,
          0.5475    ,  0.62858   ,  0.53228   ,  0.53229   ,  0.5475    ,
          0.62858   ,  0.45305768, -0.13823013,  3.8165    ,  2.179     ]])]

In [23]:
# Example SHAP Analysis
# Imports
from chemprop.args import TrainArgs
from chemprop.models.model import MoleculeModel

import torch
from chemprop.utils import load_checkpoint, load_scalers
from chemprop.features import set_extra_bond_fdim

# Load TrainArgs and MoleculeModel
path = "/home/oscarwu/code/chemprop_developing/developing/shap_v1/data/model.pt"
state = torch.load(path, map_location=lambda storage, loc: storage)
train_args = TrainArgs()
train_args.from_dict(vars(state["args"]), skip_unsettable=True)
set_extra_bond_fdim(train_args.bond_features_size)
model = load_checkpoint(path)
scalers = load_scalers(path)

# Create a MoleculeModel Wrapper
class MoleculeModelWrapper:
    def __init__(self, model, train_args, scalers, smiles, shap, features_batch, 
                 atom_descriptors_batch, atom_features_batch, bond_descriptors_batch, bond_features_batch):
        self.model = model
        self.train_args = train_args
        self.target_scaler, self.mol_feature_scaler, _, _, _ = scalers
        self.smiles = smiles
        self.shap = shap
        self.features_batch = self.mol_feature_scaler.transform(features_batch) if features_batch is not None else None
        self.atom_descriptors_batch = atom_descriptors_batch
        self.atom_features_batch = atom_features_batch
        self.bond_descriptors_batch = bond_descriptors_batch
        self.bond_features_batch = bond_features_batch
        

    def __call__(self, feature_choice):
        batch = [[smi] for smi in self.smiles]
        extra_keep_features_batch = [feature_choice[0:20]] # 20 extra molecular features
        extra_atom_keep_descriptors_batch = [feature_choice[20:33]] # 13 atom descriptors
        extra_bond_keep_descriptors_batch = None # 0 bond descriptors
        extra_atom_keep_features_batch = None # 0 atom features
        extra_bond_keep_features_batch = [feature_choice[33:37]] # 4 bond features
        chemprop_atom_keep_features = [feature_choice[37:45]] # 8 chemprop atom features
        chemprop_bond_keep_features = [feature_choice[45:49]] # 4 chemprop bond features
        
        output = self.model(batch, 
                            self.features_batch, 
                            self.atom_descriptors_batch, 
                            self.atom_features_batch, 
                            self.bond_descriptors_batch, 
                            self.bond_features_batch, 
                            shap, 
                            extra_keep_features_batch, 
                            extra_atom_keep_descriptors_batch, 
                            extra_bond_keep_descriptors_batch, 
                            extra_atom_keep_features_batch, 
                            extra_bond_keep_features_batch, 
                            chemprop_atom_keep_features, 
                            chemprop_bond_keep_features)
        
        return self.target_scaler.inverse_transform(output).flatten()
        

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.weight".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".
